In [3]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")

True

In [4]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name=os.getenv("AWS_BEDROCK_REGION"),
    aws_access_key_id=os.getenv("AWS_BEDROCK_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_BEDROCK_SECRET_ACCESS_KEY"),
)

In [5]:
def retry(times: int = 3):
    def wrap(fn):
        def inner(*args, **kwargs):
            for i in range(times):
                try:
                    return fn(*args, **kwargs)
                except Exception:
                    if i == times - 1:
                        raise
        return inner
    return wrap

In [8]:
from pathlib import Path
import json
import random
from concurrent.futures import ThreadPoolExecutor

# tiny retry decorator

images_dir = Path("data/images")
out_path = Path("data/image-generation-requests.json")
flavors = ["fairy_tale", "thriller", "romance", "science_fiction"]

@retry(3)
def make_record(idx: int, p: Path):
    b = p.read_bytes()
    fmt = "png" if p.suffix.lower() == ".png" else "jpeg"
    system = (
        "You write ultra-short, concrete instructions for a future story. "
        "One sentence, <= 15 words, imperative, grounded in the image."
    )
    user = (
        "From this image, produce a concise instruction to guide a short story later. "
        "Focus on visible elements and mood."
    )
    messages = [
        {"role": "system", "content": [{"text": system}]},
        {
            "role": "user",
            "content": [
                {"text": user},
                {"image": {"format": fmt, "source": {"bytes": b}}},
            ],
        },
    ]
    resp = llm.invoke(messages)
    ctx = str(getattr(resp, "content", "")).strip()
    return {
        "imagePath": str(p),
        "request": {
            "flavor": flavors[idx % len(flavors)],
            "additionalContext": ctx,
            "eightingPlusEnabled": random.random() < 0.3,
        },
    }

images = sorted(images_dir.glob("*"))
with ThreadPoolExecutor(max_workers=20) as pool:
    futures = [pool.submit(make_record, i, p) for i, p in enumerate(images)]
    records = [f.result() for f in futures]

out_path.write_text(json.dumps(records, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"Wrote {len(records)} records to {out_path}")

Wrote 21 records to data/image-generation-requests.json
